In [1]:
library(readxl)
library(tidyverse)
library(reshape2)


#reading in file
proteomics_df <- data.frame(read_excel("011023_Processed_Proteomics_Data.xlsx"))
head(proteomics_df)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




,Protein_Accession,Gene_Name,Protein_Descriptions,Protein_Name,Number_Unique_Peptides,Subject_ID,Sample_ID,Intensity,Imputed_Norm_Intensity,Subject_No,PrevPost
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_1,Post_1,64819128,73788697,1,Post
2,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_1,Pre_2,88131664,81867431,1,Pre
3,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_2,Post_3,30396704,32521079,2,Post
4,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_2,Pre_4,38267072,69466046,2,Pre
5,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R_16,Post_5,68394088,67261697,16,Post
6,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R_16,Pre_6,50484084,54090668,16,Pre


In [2]:
proteomics_df = proteomics_df %>%
    # just want a column that specifies the treatment and exposure separately
  separate(Subject_ID, c("R_NR", "ID"), sep = "_") %>%   
  separate(Sample_ID, c("Time_Point", "ID_2"), sep = "_") 

head(proteomics_df)

,Protein_Accession,Gene_Name,Protein_Descriptions,Protein_Name,Number_Unique_Peptides,R_NR,ID,Time_Point,ID_2,Intensity,Imputed_Norm_Intensity,Subject_No,PrevPost
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,1,Post,1,64819128,73788697,1,Post
2,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,1,Pre,2,88131664,81867431,1,Pre
3,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,2,Post,3,30396704,32521079,2,Post
4,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,2,Pre,4,38267072,69466046,2,Pre
5,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R,16,Post,5,68394088,67261697,16,Post
6,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R,16,Pre,6,50484084,54090668,16,Pre


In [9]:
proteins <- unique(proteomics_df$Protein_Accession)
#proteins list will hold unique protein accession names from Protein_Accession column of the proteomics_df

values_df <- data.frame()
#create an empty dataframe for the loop function to place values into

for (i in 1:length(proteins)) {
  #head of for loop  defines a collection of objects such as the elements of a vector or a list
  
  #we are specifying within the head of the for-loop that we want to run through a vector      containing elements from the first element [1], through the entire length of the proteins vector
    
     # unexposed df
    unexposed_df = proteomics_df %>%
        filter(Time_Point == "Pre", Protein_Accession == proteins[i])
    # exposed df
    exposed_df = proteomics_df %>%
        filter(Time_Point == "Post", Protein_Accession == proteins[i])

    # t test
    t_test = t.test(unexposed_df$Imputed_Norm_Intensity, exposed_df$Imputed_Norm_Intensity)
    
    # calculating FC to get directionality
    FC = log2(mean(exposed_df$Imputed_Norm_Intensity)/mean(unexposed_df$Imputed_Norm_Intensity))

    # contains ??? u stat, and p value
    values_vector = cbind(proteins[i], FC, t_test$statistic, t_test$p.value)
    values_df = rbind(values_df, values_vector)
}    

values_df$p.adj <- p.adjust(values_df$V4, method = "BH")

#empty_df <-  empty_df[,c(2, 5, 1, 3, 4)]
head(values_df)

,V1,FC,V3,V4,p.adj
,<chr>,<chr>,<chr>,<chr>,<dbl>
t,P61626,-0.0728486146255378,0.585411887241969,0.560815096066013,0.7958692
t1,P01877,0.212111161818743,-1.17593644942288,0.245514975976604,0.6021921
t2,P01876,0.148517136427489,-0.982069487757299,0.33080863361061,0.6587227
t3,P02768,-0.16409331368835,0.678951578641631,0.500869057582336,0.7684498
t4,P01834,0.014972216724404,-0.108493302294183,0.914024918562897,0.9551508
t5,Q8TDL5,-0.132675814878414,0.590566484071262,0.557377293224984,0.7958692


In [14]:
values_df %>%
    filter(p.adj < 0.1 & FC < 0)

V1,FC,V3,V4,p.adj
<chr>,<chr>,<chr>,<chr>,<dbl>


In [ ]:
#make a new data frame that looks within the proteomics_df and the filter function will retain all rows that satisfy the conditions when the Protein_Accession variable is the same as the variable from the proteins list.
  #The i in proteins[i] is specify that the loop filters through this filter step through the entirety of the proteins list
  
#   pre <- filtered %>%
#     filter(pre_post == "Pre") 
  
# #Now we will make another data frame, we will filter for rows from the 'filtered' df when the pre_post variable is "Pre"
#   #In this step, we also want to specify that the intensity values from this new 'Pre' filtered data frame are 'pre'... so we will rename the Intensity column in this df to 'Intesity_Pre'
  
#   post <- filtered %>%
#    filter(pre_post == "Post")
  
  
  # pre_and_post <- pre %>%
  #   right_join(post, by = "ID")
  
  # head(pre_and_post)
    #Now that the intensity has been filtered and renamed for either pre or post values, we will join these data frames together using right_join() To include all the rows of data frame 'pre' and only those from 'post' that match by "ID", specify y=TRUE.
    
#making a df for columns related to information about the proteins
   protein_info <- filtered %>% 
     select(Protein_Accession, Protein_Name, Protein_Descriptions)
     
   protein_info <- protein_info[!duplicated(protein_info), ]
   
#I only need one row of description for each protein, so I deleted the duplicate rows
  
   head(protein_info)

  ttes_res <- t.test(pre_and_post$Intensity_Pre, pre_and_post$Intensity_Post, paired = TRUE)
#from our new pre_and_post data frame, we can finally do t-tests because the pre and post values are stratified.
  #The t.test function generates a long and informative list, we can subset for specific information from this list by setting it as a ttes_res object and selecting for values like p.value

  pval <- ttes_res$p.value
  #pval object is a list that selects for the p.values generated from the ttes_res object
  
  Protein_Accession <- proteins[i] 
   
  res <- data.frame(pval, Protein_Accession)
  
  #This new object 'res' will be a data frame that contains the list object 'pval' where the Protein_Accession for this data frame will be from the entire characterized vector 'proteins'
  
  res2 <- merge(res, protein_info, by = "Protein_Accession") 
#Let's join our res data frame with the protein_info data frame     
  
  head(res2)
  
  empty_df <- rbind(empty_df, res2)
#Lastly, we will merge the res2 data frame with our empty data frame "empty_df" that we made before starting the loop
 
  
}

In [10]:
empty_df$p.adj <- p.adjust(empty_df$pval, method = "BH")

empty_df <-  empty_df[,c(2, 5, 1, 3, 4)]
head(empty_df)

,pval,p.adj,Protein_Accession,Protein_Name,Protein_Descriptions
,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,0.4518037,0.6666139,P61626,LYSC_HUMAN,Lysozyme C
2,0.1072175,0.3487203,P01877,IGHA2_HUMAN,Immunoglobulin heavy constant alpha 2
3,0.1933458,0.4515158,P01876,IGHA1_HUMAN,Immunoglobulin heavy constant alpha 1
4,0.4300396,0.6594605,P02768,ALBU_HUMAN,Albumin
5,0.8893501,0.9329493,P01834,IGKC_HUMAN,Immunoglobulin kappa constant
6,0.3978241,0.6333211,Q8TDL5,BPIB1_HUMAN,BPI fold-containing family B member 1


In [15]:
values_df = data.frame()
for (i in 1:length(proteins)){
    pre_df = proteomics_df %>%
        filter(pre_post == "Pre", Protein_Accession == proteins[i])
    post_df = proteomics_df %>%
        filter(pre_post == "Post", Protein_Accession == proteins[i])
    
    t_test = t.test(pre_df$Imputed_Norm_Intensity, post_df$Imputed_Norm_Intensity, paired = TRUE)
    values_vector = cbind(proteins[i], t_test$statistic, t_test$p.value)
    
    values_df = rbind(values_df, values_vector)
}

head(values_df)

,V1,V2,V3
,<chr>,<chr>,<chr>
t,P61626,0.763900384218391,0.45180368319395
t1,P01877,-1.66849627654674,0.107217512843925
t2,P01876,-1.33531905511343,0.193345800152901
t3,P02768,0.801618323692519,0.430039553680159
t4,P01834,-0.140496993367947,0.889350148622099
t5,Q8TDL5,0.859678075417709,0.397824079015538


# Take 2

In [7]:
dim(proteomics_df)

[1] 100116     11

In [8]:
length(unique(proteomics_df$Protein_Accession))

[1] 1854

In [9]:
length(unique(proteomics_df$Subject_ID))

[1] 27

In [11]:
(dim(proteomics_df)[1]/2)/27

[1] 1854